# Homework - Data Transformation
by `Suppawit Thiamhong`

# NYC Flight Analysis

In [1]:
install.packages("nycflights13")

library(tidyverse)
library(nycflights13)

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
Warning message:
“Failed to locate timezone database”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.5     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.0.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()  masks stats::filter()
✖ purrr::flatten() masks jsonlite::flatten()
✖ dplyr::lag()     masks stats::lag()



In [2]:
flights <- nycflights13::flights
airlines <- nycflights13::airlines
airports <- nycflights13::airports
planes <- nycflights13::planes
weather <- nycflights13::weather

In [3]:
glimpse(flights)        # glimpse data
cat("\n")
glimpse(airlines)
cat("\n")
glimpse(airports)
cat("\n")
glimpse(planes)
cat("\n")
glimpse(weather)

Rows: 336,776
Columns: 19
$ year           <int> 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2…
$ month          <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ day            <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ dep_time       <int> 517, 533, 542, 544, 554, 554, 555, 557, 557, 558, 558, …
$ sched_dep_time <int> 515, 529, 540, 545, 600, 558, 600, 600, 600, 600, 600, …
$ dep_delay      <dbl> 2, 4, 2, -1, -6, -4, -5, -3, -3, -2, -2, -2, -2, -2, -1…
$ arr_time       <int> 830, 850, 923, 1004, 812, 740, 913, 709, 838, 753, 849,…
$ sched_arr_time <int> 819, 830, 850, 1022, 837, 728, 854, 723, 846, 745, 851,…
$ arr_delay      <dbl> 11, 20, 33, -18, -25, 12, 19, -14, -8, 8, -2, -3, 7, -1…
$ carrier        <chr> "UA", "UA", "AA", "B6", "DL", "UA", "B6", "EV", "B6", "…
$ flight         <int> 1545, 1714, 1141, 725, 461, 1696, 507, 5708, 79, 301, 4…
$ tailnum        <chr> "N14228", "N24211", "N619AA", "N804JB", "N668DN", "N394…
$ origin      

# Q1: Which airlines had the least departure delay in Summer 2013?

In [11]:
flights %>%
    filter(year == 2013, month >= 6 & month <= 9 ) %>%  # filter month in summer (Jun-Sep)
    select(carrier, dep_delay) %>%
    arrange(dep_delay) %>%                              # sort by dep_delay in ascending order
    head(5) %>%                                         # top 5 airlines with the least departure delay
    left_join(airlines, by = "carrier")                 # join airlines table to get a full name of each airline

carrier,dep_delay,name
<chr>,<dbl>,<chr>
MQ,-26,Envoy Air
AA,-24,American Airlines Inc.
B6,-23,JetBlue Airways
B6,-23,JetBlue Airways
MQ,-22,Envoy Air


# Q2: Which airlines had the highest rate of canceled flights in 2013?

In [10]:
cancel_flight <- flights %>%                                # a data frame showing the number of canceled flights
  filter(is.na(dep_time)) %>%                               # filter only canceled flights
  group_by(carrier) %>%                                     # group by carrier
  summarise(canceled_flights = n())                         # count the number of canceled flights

flights %>% 
  inner_join(airlines, by = "carrier") %>%                  # join flights and airlines tables
  group_by(carrier, name) %>% 
  select(carrier, name) %>%
  summarise(scheduled_flights = n()) %>%                    # count the number of scheduled flights
  inner_join(cancel_flight, by = "carrier") %>%             # join cancel_flight table above
  mutate(canceled_flights_rate =                            # calculate the canceled flights rate
      round((canceled_flights/scheduled_flights), 4)) %>% 
  arrange(desc(canceled_flights_rate)) %>%                  # sort by the canceled flights rate in descending order
  head(5)                                                   # top 5 airlines with the higheset rate of canceled flights

`summarise()` has grouped output by 'carrier'. You can override using the `.groups` argument.



carrier,name,scheduled_flights,canceled_flights,canceled_flights_rate
<chr>,<chr>,<int>,<int>,<dbl>
OO,SkyWest Airlines Inc.,32,3,0.0938
YV,Mesa Airlines Inc.,601,56,0.0932
9E,Endeavor Air Inc.,18460,1044,0.0566
EV,ExpressJet Airlines Inc.,54173,2817,0.0520
MQ,Envoy Air,26397,1234,0.0467


# Q3: What was the total distance traveled by all airlines each month in 2013?

In [14]:
flights %>%
    filter(!is.na(dep_time)) %>%                                        # filter out all canceled flights
    inner_join(airlines, by = "carrier") %>%                            # join airlines table
    group_by(month, carrier, name) %>%                                  # group by month, carrier, and name 
    summarise(n_flights = n(),                                          # summarise the number of flights and total distance
              total_distance_in_miles = round(sum(distance), 2)) %>%
    arrange(carrier)                                                    # sort by carrier in ascending order

`summarise()` has grouped output by 'month', 'carrier'. You can override using the `.groups` argument.



month,carrier,name,n_flights,total_distance_in_miles
<int>,<chr>,<chr>,<int>,<dbl>
1,9E,Endeavor Air Inc.,1498,717534
2,9E,Endeavor Air Inc.,1353,637366
3,9E,Endeavor Air Inc.,1514,723266
4,9E,Endeavor Air Inc.,1407,691754
5,9E,Endeavor Air Inc.,1388,701809
6,9E,Endeavor Air Inc.,1276,677990
7,9E,Endeavor Air Inc.,1364,736838
8,9E,Endeavor Air Inc.,1378,748201
9,9E,Endeavor Air Inc.,1477,858655


# Q4: Which destinations were the most popular for travelers on Christmas 2013?

In [16]:
flights %>%
    filter(!is.na(dep_time), year == 2013, month == 12, day == 25) %>%  # filter date on 25 Dec
    select(year, dest) %>%
    group_by(year, dest) %>%                                            # group by year and destinations
    summarise(n_filghts = n()) %>%                                      # count the number of flights
    arrange(desc(n_filghts)) %>%                                        # sort by n_flights in descending order 
    head(5) %>%                                                         # top 5 destinations people traveled to the most
    inner_join(airports %>%                                             # join airports table to get full names of destinations
                    select(faa, name), 
               by = c("dest" = "faa"))

`summarise()` has grouped output by 'year'. You can override using the `.groups` argument.



year,dest,n_filghts,name
<int>,<chr>,<int>,<chr>
2013,MCO,41,Orlando Intl
2013,FLL,39,Fort Lauderdale Hollywood Intl
2013,ATL,37,Hartsfield Jackson Atlanta Intl
2013,LAX,36,Los Angeles Intl
2013,CLT,32,Charlotte Douglas Intl


# Q5: Which flights have the highest speed in relation to the distance traveled?

In [32]:
flights %>% 
  select(carrier, air_time, distance) %>%                   # filter carrier, air time and distance
  filter(!is.na(air_time),!is.na(distance)) %>%             # filter out all null value
  inner_join(airlines, by = "carrier") %>%                  # join airlines table for airlines name
  group_by(carrier,name) %>%                                
  mutate(velocity_in_miles_per_hour =                       # find velocity by distance/time in MPH unit
        round((distance/(air_time/60)),2)) %>%
  arrange(desc(velocity_in_miles_per_hour)) %>%             # sort velocity column in descending order
  head(10)                                                  # show top 10 flights have the highest speed

carrier,air_time,distance,name,velocity_in_miles_per_hour
<chr>,<dbl>,<dbl>,<chr>,<dbl>
DL,65,762,Delta Air Lines Inc.,703.38
EV,93,1008,ExpressJet Airlines Inc.,650.32
EV,55,594,ExpressJet Airlines Inc.,648.00
EV,70,748,ExpressJet Airlines Inc.,641.14
DL,105,1035,Delta Air Lines Inc.,591.43
DL,170,1598,Delta Air Lines Inc.,564.00
B6,172,1598,JetBlue Airways,557.44
AA,175,1623,American Airlines Inc.,556.46
DL,173,1598,Delta Air Lines Inc.,554.22
